In [12]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, LSTM
import matplotlib.pyplot as plt

import sys
sys.path.insert(0, '../utils')
import KMeansDateRange
import get_open_close
import timeseries

"""K is the number of different
industries/sectors in the stock market"""
k = 11

In [13]:
df = get_open_close.populate_values_efficient("2012-05-18", "2020-04-01")

In [14]:
clusters = KMeansDateRange.kmeans_automated("2012-05-18", "2020-04-01", k=k)

In [17]:
%%time
mean_absolute_loss = 0
"""Randomly sampling 20 stocks from each cluster"""
for cluster in range(0, k):
    sampled_stocks = clusters.loc[clusters['clusters'] == cluster].sample(n=20, random_state=1337)
    cluster_mae = 0
    
    model = Sequential()
    model.add(LSTM(128, activation='relu', input_shape=(14, 2), return_sequences=True))
    model.add(LSTM(128, activation='relu', return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=1, activation="linear"))
    model.compile(optimizer='adam', loss='mae')
    
    stock_x_test = {}
    
    """Fit the data of each stock within a cluster on the RNN"""
    for index, stock in enumerate(sampled_stocks['symbols']):
        stock_data = df[stock]
        
        x_train, x_test = train_test_split(stock_data, test_size=0.3, shuffle=False)
        
        scaler = StandardScaler()
        x_train= scaler.fit_transform(x_train)
        x_test = scaler.transform(x_test)
        stock_x_test[stock] = x_test

        x_train, y_train = timeseries.get_timeseries_X_y(x_train)
        
        model.fit(x_train, y_train, validation_split=0.3, verbose=1,
                            initial_epoch=index*20, epochs=(index + 1)*20)
        
    for stock in stock_x_test:
        x_test, y_test = timeseries.get_timeseries_X_y(stock_x_test[stock])
        loss = model.evaluate(x_test, y_test)
        cluster_mae = cluster_mae + loss
    cluster_mae = cluster_mae/(sampled_stocks.shape[0])
    mean_absolute_loss = mean_absolute_loss + cluster_mae

print("Loss of LSTM: ", mean_absolute_loss/k)

Epoch 1/20
30/30 [==============================] - 3s 37ms/step - loss: 0.3362 - val_loss: 0.1251
Epoch 2/20
30/30 [==============================] - 1s 26ms/step - loss: 0.1555 - val_loss: 0.0478
Epoch 3/20
30/30 [==============================] - 1s 25ms/step - loss: 0.1490 - val_loss: 0.0303
Epoch 4/20
30/30 [==============================] - 1s 25ms/step - loss: 0.1390 - val_loss: 0.0955
Epoch 5/20
30/30 [==============================] - 1s 24ms/step - loss: 0.1356 - val_loss: 0.1405
Epoch 6/20
30/30 [==============================] - 1s 25ms/step - loss: 0.1322 - val_loss: 0.0262
Epoch 7/20
30/30 [==============================] - 1s 25ms/step - loss: 0.1267 - val_loss: 0.0825
Epoch 8/20
30/30 [==============================] - 1s 25ms/step - loss: 0.1144 - val_loss: 0.0195
Epoch 9/20
30/30 [==============================] - 1s 25ms/step - loss: 0.1196 - val_loss: 0.0642
Epoch 10/20
30/30 [==============================] - 1s 25ms/step - loss: 0.1217 - val_loss: 0.1353
Epoch 11/

Epoch 83/100
30/30 [==============================] - 1s 28ms/step - loss: 0.0798 - val_loss: 0.1024
Epoch 84/100
30/30 [==============================] - 1s 25ms/step - loss: 0.0814 - val_loss: 0.1000
Epoch 85/100
30/30 [==============================] - 1s 25ms/step - loss: 0.0827 - val_loss: 0.1239
Epoch 86/100
30/30 [==============================] - 1s 25ms/step - loss: 0.0820 - val_loss: 0.1105
Epoch 87/100
30/30 [==============================] - 1s 25ms/step - loss: 0.0852 - val_loss: 0.1088
Epoch 88/100
30/30 [==============================] - 1s 31ms/step - loss: 0.0814 - val_loss: 0.1183
Epoch 89/100
30/30 [==============================] - 1s 27ms/step - loss: 0.0887 - val_loss: 0.1242
Epoch 90/100
30/30 [==============================] - 1s 23ms/step - loss: 0.0803 - val_loss: 0.1289
Epoch 91/100
30/30 [==============================] - 1s 24ms/step - loss: 0.0822 - val_loss: 0.1194
Epoch 92/100
30/30 [==============================] - 1s 26ms/step - loss: 0.0810 - val_los

30/30 [==============================] - 1s 27ms/step - loss: 0.0490 - val_loss: 0.1311
Epoch 164/180
30/30 [==============================] - 1s 26ms/step - loss: 0.0446 - val_loss: 0.1486
Epoch 165/180
30/30 [==============================] - 1s 25ms/step - loss: 0.0457 - val_loss: 0.1492
Epoch 166/180
30/30 [==============================] - 1s 26ms/step - loss: 0.0449 - val_loss: 0.1224
Epoch 167/180
30/30 [==============================] - 1s 26ms/step - loss: 0.0459 - val_loss: 0.1183
Epoch 168/180
30/30 [==============================] - 1s 25ms/step - loss: 0.0449 - val_loss: 0.1154
Epoch 169/180
30/30 [==============================] - 1s 27ms/step - loss: 0.0445 - val_loss: 0.1058
Epoch 170/180
30/30 [==============================] - 1s 27ms/step - loss: 0.0442 - val_loss: 0.1123
Epoch 171/180
30/30 [==============================] - 1s 25ms/step - loss: 0.0435 - val_loss: 0.1255
Epoch 172/180
30/30 [==============================] - 1s 26ms/step - loss: 0.0449 - val_loss: 0

30/30 [==============================] - 1s 24ms/step - loss: 0.1435 - val_loss: 0.0588
Epoch 244/260
30/30 [==============================] - 1s 24ms/step - loss: 0.1336 - val_loss: 0.0547
Epoch 245/260
30/30 [==============================] - 1s 25ms/step - loss: 0.1348 - val_loss: 0.0483
Epoch 246/260
30/30 [==============================] - 1s 24ms/step - loss: 0.1345 - val_loss: 0.0428
Epoch 247/260
30/30 [==============================] - 1s 23ms/step - loss: 0.1347 - val_loss: 0.0490
Epoch 248/260
30/30 [==============================] - 1s 24ms/step - loss: 0.1363 - val_loss: 0.0465
Epoch 249/260
30/30 [==============================] - 1s 24ms/step - loss: 0.1373 - val_loss: 0.0465
Epoch 250/260
30/30 [==============================] - 1s 24ms/step - loss: 0.1345 - val_loss: 0.0549
Epoch 251/260
30/30 [==============================] - 1s 24ms/step - loss: 0.1338 - val_loss: 0.0490
Epoch 252/260
30/30 [==============================] - 1s 24ms/step - loss: 0.1330 - val_loss: 0

30/30 [==============================] - 1s 30ms/step - loss: 0.0970 - val_loss: 0.0302
Epoch 324/340
30/30 [==============================] - 1s 29ms/step - loss: 0.0936 - val_loss: 0.0244
Epoch 325/340
30/30 [==============================] - 1s 28ms/step - loss: 0.0954 - val_loss: 0.0252
Epoch 326/340
30/30 [==============================] - 1s 28ms/step - loss: 0.0956 - val_loss: 0.0313
Epoch 327/340
30/30 [==============================] - 1s 28ms/step - loss: 0.0940 - val_loss: 0.0209
Epoch 328/340
30/30 [==============================] - 1s 28ms/step - loss: 0.1002 - val_loss: 0.0377
Epoch 329/340
30/30 [==============================] - 1s 28ms/step - loss: 0.0979 - val_loss: 0.0209
Epoch 330/340
30/30 [==============================] - 1s 28ms/step - loss: 0.0981 - val_loss: 0.0596
Epoch 331/340
30/30 [==============================] - 1s 30ms/step - loss: 0.0993 - val_loss: 0.0401
Epoch 332/340
30/30 [==============================] - 1s 31ms/step - loss: 0.0983 - val_loss: 0

19/19 [==============================] - 0s 13ms/step - loss: 0.0169
Loss of LSTM:  0.21610660776495932
CPU times: user 15min 59s, sys: 5min 26s, total: 21min 25s
Wall time: 5min 32s
